In [1]:
# Author: Mike Hagenow
# Generate some templated behaviors for sanding (including corrections) to be learned through DMP-LWR

import numpy as np
import rospy

from corrective_shared_autonomy.TaskModels.DMPLWRhardcoded import HybridSegment
from corrective_shared_autonomy.TaskModels.DMPLWRhardcoded import DMPLWRhardcoded
from core_robotics.PyBSpline import BSplineSurface

# starting vals is a list of starting vals
# ending vals is a list of ending vals
# num_pts is how many interp values
def interpMultD(starting_vals,ending_vals,num_pts):
    vals = []
    for ii in range(0,num_pts):
        c_i = float(ii)/float(num_pts) # interpolation coefficient
        vals.append(list(c_i*(e-s)+s for s,e in zip(starting_vals,ending_vals)))
    return np.asarray(vals).T

# returns num_vars x num_samples




Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



ImportError: dynamic module does not define module export function (PyInit__tf2)

In [6]:
# Behavior one: Three passes on the layup2 surface (for testing without running sanding tool)

surfaceBSplineFile = '/home/mike/Documents/panda_uli_workspace/src/panda_uli_demo/ULIConfig/registration_models/layup_tool2_surface_demo.csv'
surfaceBSpline = BSplineSurface()
surfaceBSpline.loadSurface(surfaceBSplineFile)

segments = []

# Part 1: approach the starting point
segment = HybridSegment()
segment.hybrid = False
segment.num_samples = 200 # 2 seconds
segment.state_names = ['x','y','z','qx','qy','qz','qw']

approach_pt, _, _, _ = surfaceBSpline.calculate_surface_point(0.4, 0.5)

starting = [approach_pt[0], approach_pt[1], approach_pt[2] + 0.05, 0.0, 0.0, 0.0, 1.0]
ending = [approach_pt[0], approach_pt[1], approach_pt[2], 0.0, 0.0, 0.0, 1.0]
orig_vals = interpMultD(starting,ending,segment.num_samples)

# number of state variables by number of samples

segment.original_vals = []
segment.original_vals.append(orig_vals)

segments.append(segment)


# Part 2: one pass over surface (for now)
segment = HybridSegment()
segment.hybrid = True
segment.surface = '/home/mike/Documents/panda_uli_workspace/src/panda_uli_demo/ULIConfig/registration_models/layup_tool2_surface_demo.csv'
segment.num_samples = 400 # 4 seconds
segment.state_names = ['u','v','f','theta_qx','theta_qy','theta_qz','theta_qw']
segment.original_vals = []

waypoint_0 = interpMultD([0.4, 0.5, 5.0, 0.0, 0.0, 0.0, 1.0],[0.9, 0.5, 5.0, 0.0, 0.0, 0.0, 1.0],segment.num_samples)
print(waypoint_0)

segment.original_vals.append(waypoint_0)

segments.append(segment)


# # Part 3: move away from the surface
segment = HybridSegment()
segment.hybrid = False
segment.num_samples = 200 # 2 seconds
segment.state_names = ['x','y','z','qx','qy','qz','qw']
segment.original_vals = []

retract_pt, _, _, _ = surfaceBSpline.calculate_surface_point(0.9, 0.5)

starting = [retract_pt[0], retract_pt[1], retract_pt[2], 0.0, 0.0, 0.0, 1.0]
ending = [retract_pt[0], retract_pt[1], retract_pt[2] + 0.05, 0.0, 0.0, 0.0, 1.0]
orig_vals = interpMultD(starting,ending,200)
segment.original_vals.append(orig_vals)

segments.append(segment)







[[0.4     0.40125 0.4025  ... 0.89625 0.8975  0.89875]
 [0.5     0.5     0.5     ... 0.5     0.5     0.5    ]
 [5.      5.      5.      ... 5.      5.      5.     ]
 ...
 [0.      0.      0.      ... 0.      0.      0.     ]
 [0.      0.      0.      ... 0.      0.      0.     ]
 [1.      1.      1.      ... 1.      1.      1.     ]]


In [7]:
############################
# Learn model              #
############################
model = DMPLWRhardcoded(verbose=True)
learnedSegments = model.learnModel(segments,'') # second argument is the outfile


-------------------
Model Learning: DMP + LWR Hardcoded
-------------------
DMP for segment  0  of  2
DMP for segment  1  of  2
DMP for segment  2  of  2
Time to learn model: 10.057498693466187 seconds


In [ ]:
############################
# Execute model            #
############################

r_surf = np.array([-0.00169263, 0.00702858, 0.70789209, 0.7062836 ])
t_surf = np.array([0.71631112, -0.17206442, 0.04937141])

model.executeModel(learnedSegments=learnedSegments, R_surface = r_surf, t_surface = t_surf)

